<a href="https://colab.research.google.com/github/krishnaaxo/NLP_Summarization_Using_Transformer/blob/main/Summarize_Stock_News_using_Python_and_Deep_Learning_from_Web_Scraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install and Import Baseline Dependencies

In [1]:
!pip install transformers

In [2]:
pip install sentencepiece

In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

# 2. Setup Summarization Model

In [4]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name) 

# 3. Summarize a Single Article

In [5]:
url = "https://au.finance.yahoo.com/news/china-restricting-tesla-use-uncovers-a-significant-challenge-for-elon-musk-expert-161921664.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [6]:
paragraphs

[<p class="Fz(14px) LineClamp(1,1.3em) M(0)">Experts share where the profits will be in the next 12 months. Don't miss this free event.</p>,
 <p>Renewed political tensions between the U.S. and China — which came to light this week as the Biden administration sat down with their Chinese counterparts for the first time to discuss a range of issues — could ensnarl vehicle maker Tesla (<a class="link rapid-noclick-resp" data-ylk="slk:TSLA" href="https://finance.yahoo.com/quote/TSLA?p=TSLA&amp;.tsrc=fin-srch">TSLA</a>), which has pushed successfully into China in recent years.</p>,
 <p>In fact, the heightened rhetoric between the two super economic superpowers may have already had blowback on Elon Musk's electric car company.</p>,
 <p>The Chinese government is restricting the use of Tesla vehicles by military staff and employees of vital state-owned companies, <a class="link rapid-noclick-resp" data-ylk="slk:The Wall Street Journal reported Friday" href="https://www.wsj.com/articles/china-t

In [7]:
paragraphs[0].text

"Experts share where the profits will be in the next 12 months. Don't miss this free event."

In [8]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [9]:
ARTICLE

'Experts share where the profits will be in the next 12 months. Don\'t miss this free event. Renewed political tensions between the U.S. and China — which came to light this week as the Biden administration sat down with their Chinese counterparts for the first time to discuss a range of issues — could ensnarl vehicle maker Tesla (TSLA), which has pushed successfully into China in recent years. In fact, the heightened rhetoric between the two super economic superpowers may have already had blowback on Elon Musk\'s electric car company. The Chinese government is restricting the use of Tesla vehicles by military staff and employees of vital state-owned companies, The Wall Street Journal reported Friday. Chinese officials reportedly have concerns that Tesla\'s cars — outfitted with various data collecting capabilities — could serve as leakers of national security secrets. "I have been saying for months now that Tesla\'s level of integration of their business, of their research, of their s

In [10]:
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')
output = model.generate(input_ids, max_length=110, num_beams=5, early_stopping=True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [11]:
summary

'Tensions between U.S. and China may have already had blowback. Tesla has been successful in China in recent years'

# 4. Building a News and Sentiment Pipeline

In [12]:
monitored_tickers = ['DOGE', 'TSLA', 'BTC']

## 4.1. Search for Stock News using Google and Yahoo Finance

In [13]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs 

In [14]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'BTC': ['/?sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQOwgC',
  '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQPAgE',
  '/search?q=yahoo+finance+BTC&tbm=nws&ie=UTF-8&gbv=1&sei=0MDEYIjPJoGv0PEP0ZiN0AQ',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQ_AUIBygA',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQ_AUICSgC',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQ_AUICigD',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQ_AUICygE',
  'https://maps.google.com/maps?q=yahoo+finance+BTC&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQ_AUIDCgF',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiI9_24o5LxAhWBFzQIHVFMA0oQ_AUIDSgG',
  '/advanced_search',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&s

In [15]:
raw_urls['DOGE']

['/?sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4QOwgC',
 '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4QPAgE',
 '/search?q=yahoo+finance+DOGE&tbm=nws&ie=UTF-8&gbv=1&sei=z8DEYIWnMZu_0PEPvPOk8AU',
 '/search?q=yahoo+finance+DOGE&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4Q_AUIBygA',
 '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4Q_AUICSgC',
 '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4Q_AUICigD',
 '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4Q_AUICygE',
 'https://maps.google.com/maps?q=yahoo+finance+DOGE&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4Q_AUIDCgF',
 '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjFysu4o5LxAhWbHzQIHbw5CV4Q_AUIDSgG',
 '/advanced_search',
 '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0a

## 4.2. Strip out unwanted URLs

In [16]:
import re

In [17]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [18]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [19]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'BTC': ['https://finance.yahoo.com/news/bitcoin-is-bad-for-crime-says-crypto-asset-manager-210045513.html',
  'https://finance.yahoo.com/news/using-macd-analyze-btc-price-111400297.html',
  'https://finance.yahoo.com/video/although-bitcoin-very-volatile-only-204141433.html',
  'https://finance.yahoo.com/news/chinas-cryptocurrency-mining-crackdown-spreads-091422332.html',
  'https://finance.yahoo.com/news/south-koreans-pay-bitcoin-stacks-222632410.html',
  'https://finance.yahoo.com/news/wannabe-bitcoin-etfs-mushrooming-getting-130000561.html',
  'https://finance.yahoo.com/news/locked-bitcoin-taproot-upgrade-gets-120837972.html',
  'https://finance.yahoo.com/news/betting-markets-see-more-pain-for-bitcoin-investors-bet-on-drop-to-10-k-oddsmakers-142311201.html',
  'https://finance.yahoo.com/news/jpmorgan-predicts-bitcoin-2021-slide-101400075.html',
  'https://finance.yahoo.com/news/bitcoin-200k-end-crypto-options-144311059.html'],
 'DOGE': ['https://finance.yahoo.com/news/dogecoin-follo

## 4.3. Search and Scrape Cleaned URLs

In [20]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [21]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'BTC': ['The Justice Department was recently able to recover $2.3 million worth of bitcoin (BTC-USD) extorted from Colonial Pipeline, suggesting that digital currency may not always be the best tool for criminals. “It’s not intuitive to a lot of people, but it\'s a thing that the public is learning more and more,” Hunter Horsley, cofounder and CEO for Bitwise Asset Management, said about the rise in recovery of stolen cryptocurrency, such as part of the bitcoin ransom paid by Colonial Pipeline to cyberhackers. Put simply, he said, "Bitcoin is bad for crime." Speaking to Yahoo Finance Live this week, Horsley pointed to data from U.S. cybersecurity analysis firm Chainalysis, which shows a downward trend in the use of cryptocurrencies by cybercriminals. “In fact, between 2019 and 2020, the occurrence of crypto use in those scenarios is down about 80%,” he said. “So I think that criminals are getting smarter that crypto is a pretty poor tool for them, and with Colonial we saw that in full

In [22]:
articles['TSLA'][2]

'Stay in the know at a glance with the Top 10 daily stories (Bloomberg) -- Shares of the world’s largest electric-vehicle battery maker hit an all-time high Thursday in the wake of a flurry of news that spurred bullish prospects for the industry. Contemporary Amperex Technology Co., a Tesla Inc. supplier, jumped as much as 8.6% in Shenzhen, taking its gains from a low in March to more than 50%. CATL led advances in not just in EVs but green energy-related stocks as well. A BloombergNEF outlook report and China’s moves to boost the sector including People’s Bank of China Governor Yi Gang pledging increased support spurred the gains. Investor appetite is growing for the Chinese company that’s been a beneficiary of a worldwide push toward curbing carbon emissions. BloombergNEF analysts expect global annual passenger EV sales rising to 14 million in 2025 from 3.1 million in 2020 and lithium-battery demand growing rapidly to approach 4.5TWh annually by 2035. This year “marks the first major

## 4.4. Summarise all Articles

In [23]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [24]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'BTC': ['‘Bitcoin is bad for crime,’ Hunter Horsley says. ‘Law enforcement is honing their investigative skills,’Trace says',
  'Bitcoin price has been given four bearish reversal signals in the last five years.',
  'China cracks down on miners using electricity. Tesla chief Elon Musk conspicuously absent at Bitcoin Mining Council',
  'Authorities in Yunnan province launch probe. China accounts for over half of global bitcoin production',
  'STX and Bitcoin will be accepted as methods of payment. Businesses that accept Paycoin will allow for customers to transact',
  'Regulators have already delayed decision on at least nine Bitcoin ETF applications. Invesco releases two ‘Bitcoin adjacent’ funds',
  'Hard fork expected to begin on Sunday. Miners have until Sunday to signal support for upgrade',
  'The virtual currency’s chances of falling below $10,000 have increased. Bitcoin has lost 40% of its market share in recent years',
  'Strategists at JPMorgan see backwardation in spot price.

In [25]:
summaries['BTC']

['‘Bitcoin is bad for crime,’ Hunter Horsley says. ‘Law enforcement is honing their investigative skills,’Trace says',
 'Bitcoin price has been given four bearish reversal signals in the last five years.',
 'China cracks down on miners using electricity. Tesla chief Elon Musk conspicuously absent at Bitcoin Mining Council',
 'Authorities in Yunnan province launch probe. China accounts for over half of global bitcoin production',
 'STX and Bitcoin will be accepted as methods of payment. Businesses that accept Paycoin will allow for customers to transact',
 'Regulators have already delayed decision on at least nine Bitcoin ETF applications. Invesco releases two ‘Bitcoin adjacent’ funds',
 'Hard fork expected to begin on Sunday. Miners have until Sunday to signal support for upgrade',
 'The virtual currency’s chances of falling below $10,000 have increased. Bitcoin has lost 40% of its market share in recent years',
 'Strategists at JPMorgan see backwardation in spot price. They warn of re

In [26]:
summaries['DOGE']

['Vitalik Buterin has become a rock star in the crypto community. Dogecoin has lost about one-quarter of its value in past week',
 'Coin is most popular ticker on Yahoo Finance over the last month.',
 'It’s up more than 14,000% in past year, making it sixth-biggest cryptocurrency. Dogecoin launched on Coinbase, making it more accessible',
 'Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats',
 'Hertz, U-Haul all had similar fortunes. SPACs, crypto coins could be ‘huge winners,’ Colas says',
 'Dogecoin rallied to a late afternoon intraday high $0.3549. Reversing a 1.63% loss from Tuesday, Dogecoin ended the day at $0.3431',
 'Vanda says retail buying may have peaked this week. Cinema chain AMC has warned investors not to buy its stock',
 'iTrustCapital now allows people to invest in the asset. Elon Musk has been known to prop the coin on his social media accounts',
 'Is Dogecoin worth $36,000-plus asking price? Decentralized, it’s not 

In [27]:
summaries['TSLA']

['Cathie Wood believes that TSLA stock can touch $3,000 by FY2025. Manufacturing presence, China, India catalysts likely to boost earnings',
 'Guillen was one of four top executives running the company. Executive previously served as president of Tesla’s automotive business',
 'CATL leads advances in EV, green energy-related stocks. Apple in early talks with Chinese firms on EV supply',
 'Tesla says it delivered 33,000 cars in China in June.',
 'Li’s Nio is targeting premium buyers with ‘Rippling Mode’. China’s luxury electric vehicle market is booming',
 "Cohen's company is rolling out a fleet of Tesla Model Y taxis.",
 'Guillen was named president of the Tesla Heavy Trucking unit in March.',
 'Guillen was named president of the Tesla Heavy Trucking unit in March.',
 'Elon Musk tweets that Model S Plaid is adequate. Model S Plaid Plus was due to hit the market by mid-2022',
 'Asia-based company is betting on U.S. demand. Swapping stations are a key part of EV transition']

# 5. Adding Sentiment Analysis

In [28]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

In [29]:
sentiment(summaries['BTC'])

[{'label': 'NEGATIVE', 'score': 0.9627186059951782},
 {'label': 'NEGATIVE', 'score': 0.9975571632385254},
 {'label': 'NEGATIVE', 'score': 0.999688446521759},
 {'label': 'NEGATIVE', 'score': 0.9684471487998962},
 {'label': 'POSITIVE', 'score': 0.9654040336608887},
 {'label': 'NEGATIVE', 'score': 0.9960678815841675},
 {'label': 'NEGATIVE', 'score': 0.977043867111206},
 {'label': 'NEGATIVE', 'score': 0.9994444251060486},
 {'label': 'NEGATIVE', 'score': 0.9988926649093628},
 {'label': 'NEGATIVE', 'score': 0.9962055683135986}]

In [30]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'BTC': [{'label': 'NEGATIVE', 'score': 0.9627186059951782},
  {'label': 'NEGATIVE', 'score': 0.9975571632385254},
  {'label': 'NEGATIVE', 'score': 0.999688446521759},
  {'label': 'NEGATIVE', 'score': 0.9684471487998962},
  {'label': 'POSITIVE', 'score': 0.9654040336608887},
  {'label': 'NEGATIVE', 'score': 0.9960678815841675},
  {'label': 'NEGATIVE', 'score': 0.977043867111206},
  {'label': 'NEGATIVE', 'score': 0.9994444251060486},
  {'label': 'NEGATIVE', 'score': 0.9988926649093628},
  {'label': 'NEGATIVE', 'score': 0.9962055683135986}],
 'DOGE': [{'label': 'NEGATIVE', 'score': 0.9957894682884216},
  {'label': 'NEGATIVE', 'score': 0.866360604763031},
  {'label': 'POSITIVE', 'score': 0.9972788691520691},
  {'label': 'POSITIVE', 'score': 0.9989463090896606},
  {'label': 'NEGATIVE', 'score': 0.9281707406044006},
  {'label': 'NEGATIVE', 'score': 0.9952418208122253},
  {'label': 'NEGATIVE', 'score': 0.9883205890655518},
  {'label': 'POSITIVE', 'score': 0.974581241607666},
  {'label': 'NEG

In [31]:
print(summaries['DOGE'][3], scores['DOGE'][3]['label'], scores['DOGE'][3]['score'])

Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats POSITIVE 0.9989463090896606


In [32]:
scores['BTC'][0]['score']

0.9627186059951782

# 6. Exporting Results to CSV

In [43]:
summaries

{'BTC': ['‘Bitcoin is bad for crime,’ Hunter Horsley says. ‘Law enforcement is honing their investigative skills,’Trace says',
  'Bitcoin price has been given four bearish reversal signals in the last five years.',
  'China cracks down on miners using electricity. Tesla chief Elon Musk conspicuously absent at Bitcoin Mining Council',
  'Authorities in Yunnan province launch probe. China accounts for over half of global bitcoin production',
  'STX and Bitcoin will be accepted as methods of payment. Businesses that accept Paycoin will allow for customers to transact',
  'Regulators have already delayed decision on at least nine Bitcoin ETF applications. Invesco releases two ‘Bitcoin adjacent’ funds',
  'Hard fork expected to begin on Sunday. Miners have until Sunday to signal support for upgrade',
  'The virtual currency’s chances of falling below $10,000 have increased. Bitcoin has lost 40% of its market share in recent years',
  'Strategists at JPMorgan see backwardation in spot price.

{'BTC': ['‘Bitcoin is bad for crime,’ Hunter Horsley says. ‘Law enforcement is honing their investigative skills,’Trace says',
  'Bitcoin price has been given four bearish reversal signals in the last five years.',
  'China cracks down on miners using electricity. Tesla chief Elon Musk conspicuously absent at Bitcoin Mining Council',
  'Authorities in Yunnan province launch probe. China accounts for over half of global bitcoin production',
  'STX and Bitcoin will be accepted as methods of payment. Businesses that accept Paycoin will allow for customers to transact',
  'Regulators have already delayed decision on at least nine Bitcoin ETF applications. Invesco releases two ‘Bitcoin adjacent’ funds',
  'Hard fork expected to begin on Sunday. Miners have until Sunday to signal support for upgrade',
  'The virtual currency’s chances of falling below $10,000 have increased. Bitcoin has lost 40% of its market share in recent years',
  'Strategists at JPMorgan see backwardation in spot price.

In [44]:
scores

{'BTC': [{'label': 'NEGATIVE', 'score': 0.9627186059951782},
  {'label': 'NEGATIVE', 'score': 0.9975571632385254},
  {'label': 'NEGATIVE', 'score': 0.999688446521759},
  {'label': 'NEGATIVE', 'score': 0.9684471487998962},
  {'label': 'POSITIVE', 'score': 0.9654040336608887},
  {'label': 'NEGATIVE', 'score': 0.9960678815841675},
  {'label': 'NEGATIVE', 'score': 0.977043867111206},
  {'label': 'NEGATIVE', 'score': 0.9994444251060486},
  {'label': 'NEGATIVE', 'score': 0.9988926649093628},
  {'label': 'NEGATIVE', 'score': 0.9962055683135986}],
 'DOGE': [{'label': 'NEGATIVE', 'score': 0.9957894682884216},
  {'label': 'NEGATIVE', 'score': 0.866360604763031},
  {'label': 'POSITIVE', 'score': 0.9972788691520691},
  {'label': 'POSITIVE', 'score': 0.9989463090896606},
  {'label': 'NEGATIVE', 'score': 0.9281707406044006},
  {'label': 'NEGATIVE', 'score': 0.9952418208122253},
  {'label': 'NEGATIVE', 'score': 0.9883205890655518},
  {'label': 'POSITIVE', 'score': 0.974581241607666},
  {'label': 'NEG

{'BTC': [{'label': 'NEGATIVE', 'score': 0.9627186059951782},
  {'label': 'NEGATIVE', 'score': 0.9975571632385254},
  {'label': 'NEGATIVE', 'score': 0.999688446521759},
  {'label': 'NEGATIVE', 'score': 0.9684471487998962},
  {'label': 'POSITIVE', 'score': 0.9654040336608887},
  {'label': 'NEGATIVE', 'score': 0.9960678815841675},
  {'label': 'NEGATIVE', 'score': 0.977043867111206},
  {'label': 'NEGATIVE', 'score': 0.9994444251060486},
  {'label': 'NEGATIVE', 'score': 0.9988926649093628},
  {'label': 'NEGATIVE', 'score': 0.9962055683135986}],
 'DOGE': [{'label': 'NEGATIVE', 'score': 0.9957894682884216},
  {'label': 'NEGATIVE', 'score': 0.866360604763031},
  {'label': 'POSITIVE', 'score': 0.9972788691520691},
  {'label': 'POSITIVE', 'score': 0.9989463090896606},
  {'label': 'NEGATIVE', 'score': 0.9281707406044006},
  {'label': 'NEGATIVE', 'score': 0.9952418208122253},
  {'label': 'NEGATIVE', 'score': 0.9883205890655518},
  {'label': 'POSITIVE', 'score': 0.974581241607666},
  {'label': 'NEG

In [45]:
cleaned_urls

{'BTC': ['https://finance.yahoo.com/news/bitcoin-is-bad-for-crime-says-crypto-asset-manager-210045513.html',
  'https://finance.yahoo.com/news/using-macd-analyze-btc-price-111400297.html',
  'https://finance.yahoo.com/video/although-bitcoin-very-volatile-only-204141433.html',
  'https://finance.yahoo.com/news/chinas-cryptocurrency-mining-crackdown-spreads-091422332.html',
  'https://finance.yahoo.com/news/south-koreans-pay-bitcoin-stacks-222632410.html',
  'https://finance.yahoo.com/news/wannabe-bitcoin-etfs-mushrooming-getting-130000561.html',
  'https://finance.yahoo.com/news/locked-bitcoin-taproot-upgrade-gets-120837972.html',
  'https://finance.yahoo.com/news/betting-markets-see-more-pain-for-bitcoin-investors-bet-on-drop-to-10-k-oddsmakers-142311201.html',
  'https://finance.yahoo.com/news/jpmorgan-predicts-bitcoin-2021-slide-101400075.html',
  'https://finance.yahoo.com/news/bitcoin-200k-end-crypto-options-144311059.html'],
 'DOGE': ['https://finance.yahoo.com/news/dogecoin-follo

{'BTC': ['https://finance.yahoo.com/news/bitcoin-is-bad-for-crime-says-crypto-asset-manager-210045513.html',
  'https://finance.yahoo.com/news/using-macd-analyze-btc-price-111400297.html',
  'https://finance.yahoo.com/video/although-bitcoin-very-volatile-only-204141433.html',
  'https://finance.yahoo.com/news/chinas-cryptocurrency-mining-crackdown-spreads-091422332.html',
  'https://finance.yahoo.com/news/south-koreans-pay-bitcoin-stacks-222632410.html',
  'https://finance.yahoo.com/news/wannabe-bitcoin-etfs-mushrooming-getting-130000561.html',
  'https://finance.yahoo.com/news/locked-bitcoin-taproot-upgrade-gets-120837972.html',
  'https://finance.yahoo.com/news/betting-markets-see-more-pain-for-bitcoin-investors-bet-on-drop-to-10-k-oddsmakers-142311201.html',
  'https://finance.yahoo.com/news/jpmorgan-predicts-bitcoin-2021-slide-101400075.html',
  'https://finance.yahoo.com/news/bitcoin-200k-end-crypto-options-144311059.html'],
 'DOGE': ['https://finance.yahoo.com/news/dogecoin-follo

In [46]:
range(len(summaries['DOGE']))

range(0, 10)

range(0, 10)

In [47]:
summaries['DOGE'][3]

'Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats'

'Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats'

In [48]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [49]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['DOGE',
  'Vitalik Buterin has become a rock star in the crypto community. Dogecoin has lost about one-quarter of its value in past week',
  'NEGATIVE',
  0.9957894682884216,
  'https://finance.yahoo.com/news/dogecoin-follows-bitcoin-lead-flips-135422542.html'],
 ['DOGE',
  'Coin is most popular ticker on Yahoo Finance over the last month.',
  'NEGATIVE',
  0.866360604763031,
  'https://finance.yahoo.com/news/yahoo-finance-users-love-dogecoin-morning-brief-101645082.html'],
 ['DOGE',
  'It’s up more than 14,000% in past year, making it sixth-biggest cryptocurrency. Dogecoin launched on Coinbase, making it more accessible',
  'POSITIVE',
  0.9972788691520691,
  'https://finance.yahoo.com/news/dogecoin-bolstered-coinbase-addition-musk-070126645.html'],
 ['DOGE',
  'Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats',
  'POSITIVE',
  0.9989463090896606,
  'https://finance.yahoo.com/news/doge-price-finds-green-bitcoin-134109136.html'],


[['DOGE',
  'Vitalik Buterin has become a rock star in the crypto community. Dogecoin has lost about one-quarter of its value in past week',
  'NEGATIVE',
  0.9957894682884216,
  'https://finance.yahoo.com/news/dogecoin-follows-bitcoin-lead-flips-135422542.html'],
 ['DOGE',
  'Coin is most popular ticker on Yahoo Finance over the last month.',
  'NEGATIVE',
  0.866360604763031,
  'https://finance.yahoo.com/news/yahoo-finance-users-love-dogecoin-morning-brief-101645082.html'],
 ['DOGE',
  'It’s up more than 14,000% in past year, making it sixth-biggest cryptocurrency. Dogecoin launched on Coinbase, making it more accessible',
  'POSITIVE',
  0.9972788691520691,
  'https://finance.yahoo.com/news/dogecoin-bolstered-coinbase-addition-musk-070126645.html'],
 ['DOGE',
  'Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats',
  'POSITIVE',
  0.9989463090896606,
  'https://finance.yahoo.com/news/doge-price-finds-green-bitcoin-134109136.html'],


In [50]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [51]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['DOGE',
  'Vitalik Buterin has become a rock star in the crypto community. Dogecoin has lost about one-quarter of its value in past week',
  'NEGATIVE',
  0.9957894682884216,
  'https://finance.yahoo.com/news/dogecoin-follows-bitcoin-lead-flips-135422542.html'],
 ['DOGE',
  'Coin is most popular ticker on Yahoo Finance over the last month.',
  'NEGATIVE',
  0.866360604763031,
  'https://finance.yahoo.com/news/yahoo-finance-users-love-dogecoin-morning-brief-101645082.html'],
 ['DOGE',
  'It’s up more than 14,000% in past year, making it sixth-biggest cryptocurrency. Dogecoin launched on Coinbase, making it more accessible',
  'POSITIVE',
  0.9972788691520691,
  'https://finance.yahoo.com/news/dogecoin-bolstered-coinbase-addition-musk-070126645.html'],
 ['DOGE',
  'Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats',
  'POSITIVE',
  0.9989463090896606,
  'https://finance.yahoo.com/

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['DOGE',
  'Vitalik Buterin has become a rock star in the crypto community. Dogecoin has lost about one-quarter of its value in past week',
  'NEGATIVE',
  0.9957894682884216,
  'https://finance.yahoo.com/news/dogecoin-follows-bitcoin-lead-flips-135422542.html'],
 ['DOGE',
  'Coin is most popular ticker on Yahoo Finance over the last month.',
  'NEGATIVE',
  0.866360604763031,
  'https://finance.yahoo.com/news/yahoo-finance-users-love-dogecoin-morning-brief-101645082.html'],
 ['DOGE',
  'It’s up more than 14,000% in past year, making it sixth-biggest cryptocurrency. Dogecoin launched on Coinbase, making it more accessible',
  'POSITIVE',
  0.9972788691520691,
  'https://finance.yahoo.com/news/dogecoin-bolstered-coinbase-addition-musk-070126645.html'],
 ['DOGE',
  'Musk’s ‘We Love You Elon’ hashtag is trending on Twitter. Bitcoin’s rise lifts all top-10 crypto boats',
  'POSITIVE',
  0.9989463090896606,
  'https://finance.yahoo.com/

In [52]:
import csv
with open('Final_Result.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)